In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np

In [ ]:
data_file = open("dataset.txt", "r")
lines = data_file.readlines()

letter_to_idx = {}
idx_to_letter = {}
cnt = 0
for i in range(10):
    letter_to_idx[str(i)] = len(letter_to_idx)
for line in lines:
    if "sin" in line or "cos" in line or "tan" in line:
        line = line.replace("sin", "!")
        line = line.replace("cos", "@")
        line = line.replace("tan", "#")
        cnt += 1
    
    for letter in line:
        if letter.isalpha(): letter = "$"
        if letter not in letter_to_idx:
            letter_to_idx[letter] = len(letter_to_idx)

for k,v in letter_to_idx.items():
    idx_to_letter[v] = k
random.shuffle(lines)
train_lines = lines[: int(len(lines)*0.97) ]
test_lines = lines[int(len(lines)*0.97): ]
print(len(train_lines),len(test_lines))
print(cnt)
print(letter_to_idx)
print(idx_to_letter)
print(len(lines))



970000 30000
30095
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '(': 10, '-': 11, '*': 12, '$': 13, ')': 14, '=': 15, '\n': 16, '+': 17, '@': 18, '#': 19, '!': 20}
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '(', 11: '-', 12: '*', 13: '$', 14: ')', 15: '=', 16: '\n', 17: '+', 18: '@', 19: '#', 20: '!'}
1000000


In [ ]:
tags = []
max_digit = 0
for line in lines:
    if "sin" in line or "cos" in line or "tan" in line:
        line = line.replace("sin", "!")
        line = line.replace("cos", "@")
        line = line.replace("tan", "#")
        cnt += 1
    tag_line = []
    for i, letter in enumerate(line):
        # print(letter, letter.isnumeric())
        if letter.isnumeric():
            j = i; digit = 0
            while j < len(line):
                if line[j].isnumeric(): 
                    digit += 1
                    if digit > max_digit: max_digit = digit
                else: break
                j += 1
            tag_line.append(digit)
        else: tag_line.append(0)
    tags.append(tag_line)
train_tags = tags[: int(len(tags)*0.97) ]
test_tags = tags[int(len(tags)*0.97) : ]
print(len(train_tags),len(test_tags))

970000 30000


In [ ]:
print(max_digit)
print(len(tags))
print(len(tags[100]),len(lines[100]))
for a,b in zip(test_lines, test_tags):
    if "sin" in a or "cos" in a or "tan" in a:
        a = a.replace("sin", "!")
        a = a.replace("cos", "@")
        a = a.replace("tan", "#")
    if len(a) != len(b): print(a,b)

4
1000000
14 14


In [ ]:
class LSTMPoly(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tag_size, tag_em_dim, att_dim):
        super(LSTMPoly, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.tag_embeddings = nn.Embedding(tag_size, tag_em_dim)
        self.att_embeddings = nn.Embedding(10+1, att_dim)
        self.softmax = nn.Softmax(dim=-1)

        self.lstm = nn.LSTM(embedding_dim + tag_em_dim, hidden_dim, num_layers=4, batch_first=True)
        self.classifier1 = nn.Linear(hidden_dim * 3 , vocab_size)
        # self.activation = nn.ReLU()
        # self.classifier2 = nn.Linear(64, vocab_size)

    def forward(self, inputs, tags):
        embeds = self.word_embeddings(inputs)
        tag_ems = self.tag_embeddings(tags)
        overall_embeds = torch.cat((embeds,tag_ems), dim=1)
        overall_embeds = overall_embeds.view(1, len(inputs), -1)
        lstm_out, _ = self.lstm(overall_embeds)
        lstm_out = lstm_out.squeeze(0)
        
        # embeds = embeds.view(1, len(inputs), -1)

        att_embeds = self.att_embeddings.weight
        # print(lstm_out.shape, att_embeds.shape)
        attention_logits = torch.matmul(lstm_out, att_embeds.t())
        attention_scores = self.softmax(attention_logits)
        
        attention_embeds = torch.matmul(attention_scores, att_embeds)
        attention_embeds = torch.div(attention_embeds, 10+1) #torch.Size([36, 150])
        # print(lstm_out.shape, attention_embeds.shape)
        # print((lstm_out * attention_embeds).shape)
        attended_embeds = torch.cat((lstm_out,attention_embeds,lstm_out * attention_embeds), dim=1)

        # print(attended_embeds.shape)
        # attended_embeds = attended_embeds.view(1, len(inputs), -1)

        # lstm_out, _ = self.lstm(overall_embeds)
        # print(overall_embeds.shape, lstm_out.shape)
        logits = self.classifier1(attended_embeds)
        # logits = self.activation(logits)
        # logits = self.classifier2(logits)
        logits = logits.unsqueeze(0)
        return logits, attention_logits

def line_2_tokens(line, dic):
    tokens = [ dic[letter] if not letter.isalpha() else dic["$"] for letter in line]
    return tokens

def tokens_2_lines(line, dic):
    tokens = [ dic[letter] for letter in line]
    return tokens

def line_2_att_token(line):
    tokens = [ int(letter) if letter.isnumeric() else 10 for letter in line]
    return tokens
cnt_10 = 0; cnt_num = 0;
def evaluate_dumb(model, test_lines, test_tags):
    # cnt_10 = 0; cnt_num = 0; cnt_10_correct=0; cnt_num_correct=0
    total_letters = 0
    correct_letters = 0
    total_sentence = 0
    correct_sentence = 0
    att_correct = 0
    att_total = 0
    soft = nn.Softmax(dim=-1)
    model.eval()
    with torch.no_grad():
        for i, (line, line_tags) in enumerate(zip(test_lines,test_tags)):
            
            if "sin" in line or "cos" in line or "tan" in line:
                line = line.replace("sin", "!")
                line = line.replace("cos", "@")
                line = line.replace("tan", "#")
            target_len = len(line.split("=")[1])
            tokens = line_2_tokens(line, letter_to_idx)
            
            train_tokens = tokens[:-1]; target_tokens = tokens[-target_len:]; line_tags = line_tags[:-1]
            att_target = tokens_2_lines(target_tokens, idx_to_letter)
            att_target = line_2_att_token(att_target)

            train_tokens = torch.tensor(train_tokens).to(device);# target_tokens = torch.tensor(target_tokens)
            line_tags = torch.tensor(line_tags).to(device)

            logits, att_logits = model.forward(train_tokens, line_tags)
            preds = soft(logits)
            att_preds = soft(att_logits)
            # atts = soft(att_logits)
            preds_target = preds[:,-target_len:,:]
            att_preds_target = att_preds[-target_len:,:]
            # print(len(target_tokens))
            preds_target = np.argmax(preds_target.cpu().detach().numpy(), axis=-1)[0]
            att_preds_target = np.argmax(att_preds_target.cpu().detach().numpy(), axis=-1)

            # print(line.split("=")[0])
            # print(tokens_2_lines(preds_target, idx_to_letter) )
            # print(tokens_2_lines(target_tokens, idx_to_letter) )
            # print(" ")

            # if i == 20: break

            sentence_correct = True
            # print(tokens_2_lines(preds_target, idx_to_letter))
            # print(tokens_2_lines(target_tokens, idx_to_letter))
            # print(" ")
            for p, t in zip(preds_target, target_tokens):
                if p == t: correct_letters += 1
                else: sentence_correct = False
                total_letters += 1
            total_sentence += 1
            if sentence_correct: correct_sentence += 1

            for p, t in zip(att_preds_target, att_target):
                if p == t: att_correct += 1
                # if p == 10: 
                #     cnt_10 += 1
                #     if p == t: cnt_10_correct += 1
                # else: 
                #     cnt_num += 1
                #     if p == t: cnt_num_correct += 1
            att_total += len(att_target)

            # print(att_preds_target)
            # print(att_target)
            # print( )

    # print(cnt_10, cnt_10_correct, cnt_num, cnt_num_correct)
    return total_letters, correct_letters, total_sentence, correct_sentence, att_correct, att_total



In [ ]:
evaluate_dumb(model, test_lines, test_tags)

(20-8*k)*(7*k+7)
['-', '5', '6', '*', '$', '*', '*', '2', '+', '8', '4', '*', '$', '+', '1', '4', '0', '\n']
['-', '5', '6', '*', '$', '*', '*', '2', '+', '8', '4', '*', '$', '+', '1', '4', '0', '\n']
 
(14-6*z)*(27-z)
['6', '*', '$', '*', '*', '2', '-', '1', '7', '2', '*', '$', '+', '3', '7', '8', '\n']
['6', '*', '$', '*', '*', '2', '-', '1', '7', '6', '*', '$', '+', '3', '7', '8', '\n']
 
(-7*a-7)*(a+16)
['-', '7', '*', '$', '*', '*', '2', '-', '1', '1', '5', '*', '$', '-', '1', '1', '2', '\n']
['-', '7', '*', '$', '*', '*', '2', '-', '1', '1', '9', '*', '$', '-', '1', '1', '2', '\n']
 
(x+23)*(2*x-22)
['2', '*', '$', '*', '*', '2', '+', '2', '4', '*', '$', '-', '5', '0', '6', '\n']
['2', '*', '$', '*', '*', '2', '+', '2', '4', '*', '$', '-', '5', '0', '6', '\n']
 
7*i*(i-20)
['7', '*', '$', '*', '*', '2', '-', '1', '4', '0', '*', '$', '\n']
['7', '*', '$', '*', '*', '2', '-', '1', '4', '0', '*', '$', '\n']
 
(h-13)*(8*h+28)
['8', '*', '$', '*', '*', '2', '-', '7', '6', '*', '$', '-

(762, 739, 50, 32, 739, 762)

0.8790936040870372

In [ ]:

# device = torch.device("cuda")
model_test = LSTMPoly(96, 256, len(letter_to_idx), max_digit+1, 64, 256)
# model.forward(torch.tensor([0,1]), torch.tensor([0,1]))
model_test.to(device)

LSTMPoly(
  (word_embeddings): Embedding(21, 96)
  (tag_embeddings): Embedding(5, 64)
  (att_embeddings): Embedding(11, 256)
  (softmax): Softmax(dim=-1)
  (lstm): LSTM(160, 256, num_layers=4, batch_first=True)
  (classifier1): Linear(in_features=512, out_features=64, bias=True)
  (activation): ReLU()
  (classifier2): Linear(in_features=64, out_features=21, bias=True)
)

In [ ]:


for i, (line, line_tags) in enumerate(zip(train_lines, train_tags)):
    model.train()
    if "sin" in line or "cos" in line or "tan" in line:
        line = line.replace("sin", "!")
        line = line.replace("cos", "@")
        line = line.replace("tan", "#")
    
    tokens = line_2_tokens(line, letter_to_idx)
    target_len = len(line.split("=")[1])
    
    train_tokens = tokens[:-1]; target_tokens = tokens[-target_len:]; line_tags = line_tags[:-1]
    att_target = tokens_2_lines(target_tokens, idx_to_letter)
    att_target = line_2_att_token(att_target)
    att_target = torch.tensor(att_target).to(device)

    train_tokens = torch.tensor(train_tokens).to(device); target_tokens = torch.tensor(target_tokens).to(device)
    line_tags = torch.tensor(line_tags).to(device)
    logits, att_logits = model_test.forward(train_tokens, line_tags)
    # preds = soft(logits)
            # atts = soft(att_logits)
    att_logits = att_logits[-target_len:,:]
    # preds_target = preds[:,-target_len:,:]
    # print(att_logits.shape, target_tokens.shape)
    # print(att_logits.shape, att_target.shape)
    a = nn.CrossEntropyLoss()(att_logits, att_target)
    # print(a)
    

    break

torch.Size([28, 256]) torch.Size([28, 256])
torch.Size([28, 256])


In [ ]:
A = torch.rand(2, 3,1)# torch.tensor([[1,0],[0,1]])
B = torch.rand(2, 3, 2) #torch.tensor( [ [1,2,3] , [4, 5, 6] ] )
# A = A.unsqueeze(1)
print(A.shape, B.shape)
# C = A * B
C = torch.multiply(A,B)
print(C.shape)

torch.Size([2, 3, 1]) torch.Size([1, 3, 2])
torch.Size([2, 3, 2])


In [ ]:
device = torch.device("cuda")
model = LSTMPoly(96, 256, len(letter_to_idx), max_digit+1, 64, 256)
# model.forward(torch.tensor([0,1]), torch.tensor([0,1]))
model.to(device)
weights_main = [3.0] * 10 + [1.0] * 11
class_weights_main = torch.FloatTensor(weights_main).to(device)
loss_function = nn.CrossEntropyLoss(class_weights_main) #nn.NLLLoss()
weights = [1.0] * 10 + [0.06]
class_weights = torch.FloatTensor(weights).to(device)
att_loss_func = nn.CrossEntropyLoss(class_weights) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i, (line, line_tags) in enumerate(zip(train_lines, train_tags)):
    model.train()
    if "sin" in line or "cos" in line or "tan" in line:
        line = line.replace("sin", "!")
        line = line.replace("cos", "@")
        line = line.replace("tan", "#")
    
    tokens = line_2_tokens(line, letter_to_idx)
    target_len = len(line.split("=")[1])
    
    train_tokens = tokens[:-1]; target_tokens = tokens[-target_len:]; line_tags = line_tags[:-1]
    att_target = tokens_2_lines(target_tokens, idx_to_letter)
    att_target = line_2_att_token(att_target)
    

    train_tokens = torch.tensor(train_tokens).to(device); target_tokens = torch.tensor(target_tokens).to(device)
    line_tags = torch.tensor(line_tags).to(device); att_target = torch.tensor(att_target).to(device)

    logits, att_logits = model.forward(train_tokens, line_tags)
    logits = logits[:,-target_len:,:]
    att_logits = att_logits[-target_len:,:]
    loss = loss_function(logits.squeeze(0), target_tokens.squeeze(0))
    att_loss = att_loss_func(att_logits, att_target)

    total_loss = loss + 0.1 * att_loss

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    if i % 60000 == 0: 
        total_letters, correct_letters, total_sentence, correct_sentence, att_correct, att_total = evaluate_dumb(model, test_lines, test_tags)
        print(i, total_letters, correct_letters, correct_letters/total_letters, total_sentence, correct_sentence, correct_sentence/total_sentence, att_correct/att_total )

0 459606 13959 0.03037166616623804 30000 0 0.0 0.11919122030608825
60000 459606 375822 0.8177047297032676 30000 4863 0.1621 0.8626780329238521
120000 459606 412493 0.8974926349960618 30000 10205 0.3401666666666667 0.9183169932507409
180000 459606 428124 0.9315021997101866 30000 12035 0.40116666666666667 0.9367110089946606
240000 459606 433961 0.9442022079781378 30000 13319 0.4439666666666667 0.9475746617755207
300000 459606 436570 0.9498788092409586 30000 14207 0.4735666666666667 0.9530532673637855
360000 459606 437307 0.951482356627198 30000 14465 0.4821666666666667 0.9549831812465459
420000 459606 438904 0.9549570719268243 30000 15337 0.5112333333333333 0.957831272872852
480000 459606 440133 0.9576311014216524 30000 15856 0.5285333333333333 0.9596676283599431
540000 459606 441095 0.9597241985526733 30000 16370 0.5456666666666666 0.9626462665848574
600000 459606 441554 0.9607228800320274 30000 16462 0.5487333333333333 0.9635187530188901


In [ ]:
torch.save(model.state_dict(), "model_path")


In [ ]:
pytorch_total_params = sum(p.numel() for p in model_test.parameters() ) #if p.requires_grad)
print(pytorch_total_params)

2046389


In [ ]:
# layer 3
0 458349 46074 0.10052165489615991 30000 0 0.0 0.5928713709422296
60000 458349 381974 0.8333693321028299 30000 3174 0.1058 0.8352107237061714
120000 458349 402954 0.8791423129536663 30000 7576 0.25253333333333333 0.8793386698781932
180000 458349 414202 0.9036825650323226 30000 10038 0.3346 0.9014091881950217
240000 458349 420016 0.916367222356763 30000 10856 0.36186666666666667 0.9149338168077163
300000 458349 425010 0.9272628499244026 30000 11611 0.38703333333333334 0.9269508605887653
360000 458349 427463 0.9326146669895647 30000 11988 0.3996 0.9335942698685936
420000 458349 430899 0.9401111380192823 30000 12691 0.4230333333333333 0.9400827753524061
480000 458349 431674 0.9418019893138199 30000 12825 0.4275 0.9418718051092072
540000 458349 432263 0.9430870362976683 30000 12800 0.4266666666666667 0.9435102945572043
600000 458349 432712 0.9440666391766972 30000 13187 0.43956666666666666 0.9449153374393748
660000 458349 434135 0.9471712603278288 30000 13630 0.4543333333333333 0.9478519643328556
720000 458349 433858 0.946566917349007 30000 13530 0.451 0.9474112521244729
780000 458349 435215 0.9495275434221521 30000 13983 0.4661 0.949302823830749
---------------------------------------------------------------------------